In [2]:
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.datasets import boston_housing

import numpy as np
import plotly.graph_objects as go

In [3]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [4]:
train_data.shape, train_targets.shape

((404, 13), (404,))

In [5]:
test_data.shape, test_targets.shape

((102, 13), (102,))

In [6]:
# veri üzerinde feature-wise normalization uygulayalım (x-x_mean)/x_std şeklinde:
train_mean = train_data.mean()
train_std = train_data.std()
train_data = (train_data - train_mean) / train_std

# test verisi üzerine de trainde elde edilen verileri kullanıyoruz.
# hiç bir workflowda test üzerinde hesaplanan sayısal değerler kullanılmamalıdır. 
# kullanılması modelin test verisine yönelik çıkarım yapma potansiyelini artırır, dolayısıyla test verisi modelin tam anlamıyla "görmediği" veri olmaktan çıkar.
test_data = (test_data - train_mean) / train_std

In [7]:
# build the network in the function
def build_model():
  model = models.Sequential([
    layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1],)),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(1)
  ])
  model.compile(
      optimizer = optimizers.RMSprop(lr=0.001),
      loss = 'mse',
      metrics = ['mae']
  )
  return model

In [9]:
# Since we have a very small dataset (~500 data points in total), using K-Fold cross validation would be the best practice to have a better model
# set constants before-hand
k = 5 # I'll use 5 folds
num_val_sample = len(train_data)//k
epochs = 500
mae_scores = []
for i in range(k):
  print('Processing fold #',i+1)
  # get the folded data
  val_data = train_data[i * num_val_sample: (i+1) * num_val_sample]
  val_targets = train_targets[i * num_val_sample: (i+1) * num_val_sample]
  # get the training data by concatenating the other parts of the data
  x_train = np.concatenate((train_data[:i * num_val_sample],train_data[(i+1) * num_val_sample:]), axis = 0)
  y_train = np.concatenate((train_targets[:i * num_val_sample],train_targets[(i+1) * num_val_sample:]), axis = 0)

  model = build_model() # get compiled model
  r = model.fit(x_train, y_train, validation_data = (val_data, val_targets), epochs = epochs, batch_size = 1, verbose = 0)
  mae_scores.append(r.history['val_mae'])

Processing fold # 1
Processing fold # 2
Processing fold # 3
Processing fold # 4
Processing fold # 5


In [10]:
# validation skorlarını grafiğe dökmeden önce her birinin içerisinde epoch sayısı büyüklüğünde listeler olduğundan
# kullanılan fold sayısı kadar trace ekleyeceğim
import plotly.express as px
color_scheme = px.colors.qualitative.Plotly

fig = go.Figure()
x = np.array(range(1,epochs+1))

fig.update_layout(xaxis_title = 'Epoch',
                  yaxis_title = 'MAE')
for i in range(k):
  fig.update_layout(title = 'MAE for Fold #%s'%(i+1))
  fig.add_trace(go.Scatter(x = x, y = mae_scores[i], name = 'mae', line=dict(color=color_scheme[i]) ))
  fig.show()
  fig.data = []


In [12]:
# get the average losses per epochs and plot them

avg_mae_loss = [
  np.mean([x[i] for x in mae_scores]) for i in range(epochs)
]

fig = go.Figure()
fig.update_layout(title = 'Average MSE per Epoch', xaxis_title = 'Epoch', yaxis_title = 'avg_mse')
fig.add_trace(go.Scatter(x = x, y = avg_mae_loss, name = 'avg_mae_loss_per_epoch'))
fig.show()

In [13]:
# use the graph results to evaluate the test data and building the model again
model = build_model()
r = model.fit(train_data, train_targets, epochs = 420, batch_size=16, verbose = 2)
# evaluate the model
model.evaluate(test_data,test_targets)

Epoch 1/420
26/26 - 0s - loss: 506.0805 - mae: 20.4611
Epoch 2/420
26/26 - 0s - loss: 331.6867 - mae: 15.4963
Epoch 3/420
26/26 - 0s - loss: 164.5458 - mae: 9.6382
Epoch 4/420
26/26 - 0s - loss: 98.8090 - mae: 7.1831
Epoch 5/420
26/26 - 0s - loss: 84.0197 - mae: 6.5091
Epoch 6/420
26/26 - 0s - loss: 75.4904 - mae: 6.1219
Epoch 7/420
26/26 - 0s - loss: 69.3774 - mae: 5.8918
Epoch 8/420
26/26 - 0s - loss: 67.4759 - mae: 5.8148
Epoch 9/420
26/26 - 0s - loss: 65.6011 - mae: 5.6278
Epoch 10/420
26/26 - 0s - loss: 65.1319 - mae: 5.6906
Epoch 11/420
26/26 - 0s - loss: 64.0804 - mae: 5.5389
Epoch 12/420
26/26 - 0s - loss: 63.7804 - mae: 5.6865
Epoch 13/420
26/26 - 0s - loss: 63.0295 - mae: 5.6053
Epoch 14/420
26/26 - 0s - loss: 63.2558 - mae: 5.4995
Epoch 15/420
26/26 - 0s - loss: 62.2454 - mae: 5.5084
Epoch 16/420
26/26 - 0s - loss: 62.1393 - mae: 5.4711
Epoch 17/420
26/26 - 0s - loss: 61.2890 - mae: 5.4890
Epoch 18/420
26/26 - 0s - loss: 61.5375 - mae: 5.5707
Epoch 19/420
26/26 - 0s - loss: 

[28.051963806152344, 3.501603126525879]